In [384]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

plt.style.use('ggplot')
%matplotlib inline

In [385]:
train = pd.read_csv('train.csv', index_col = 'PassengerId')
train['PassengerId'] = train.index.values
kaggle = pd.read_csv('test.csv')
PassengerId = kaggle['PassengerId'].tolist()




In [386]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 12 columns):
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
PassengerId    891 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 90.5+ KB


In [387]:
train.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare,PassengerId
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,-0.005007
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.035144
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.036847
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.057527
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.001652
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.012658
PassengerId,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658,1.000000


In [388]:
train['Age'] = train['Age'].astype('float').fillna(train['Age'].median())
train['Fare'] = train['Fare'].astype('float').fillna(train['Fare'].median())

kaggle['Age'] = kaggle['Age'].astype('float').fillna(kaggle['Age'].median())
kaggle['Fare'] = kaggle['Fare'].astype('float').fillna(kaggle['Fare'].median())

In [389]:
train['Age'] = train['Age'].astype('int')
kaggle['Age'] = kaggle['Age'].astype('int')


labels = ['+++ Survived/Age', '++ Survived/Age', '+ Survived/Age']
protection = train.groupby('Age')['Survived'].agg(['sum','count'])
protection = protection['sum']/protection['count']
protection = protection.rename('Survived/Age')
protection = pd.DataFrame(pd.qcut(protection, 3, labels = labels))

train = pd.merge(train, protection.round(1), left_on = 'Age', right_index = True)
kaggle = pd.merge(kaggle, protection.round(1), left_on = 'Age', right_index = True, how='left')
kaggle['Survived/Age'] = kaggle['Survived/Age'].fillna( '++ Survived/Age' )

survived = pd.get_dummies(train['Survived/Age'])
train = pd.merge(train, survived, left_index=True, right_index=True)

survived = pd.get_dummies(kaggle['Survived/Age'])
kaggle = pd.merge(kaggle, survived, left_index=True, right_index=True)

In [390]:
recode = {'male': 0, 'female' : 1}
train = train.replace({'Sex' : recode})
kaggle = kaggle.replace({'Sex' : recode})

mask = train['Cabin'].astype('str') == 'nan'
train['Cabin'] = mask.astype('int')

mask = kaggle['Cabin'].astype('str') == 'nan'
kaggle['Cabin'] = mask.astype('int')

In [391]:
labels = ['Pauvre', 'moyen', 'riche']

train['Fare'] = pd.qcut(train['Fare'], 3, labels = labels)
kaggle['Fare'] = pd.qcut(kaggle['Fare'], 3, labels = labels)

fare = pd.get_dummies(train['Fare'])
train = pd.merge(train, fare, left_index=True, right_index=True)

fare = pd.get_dummies(kaggle['Fare'])
kaggle = pd.merge(kaggle, fare, left_index=True, right_index=True)

In [392]:
no = 'No Title'
noble = 'Noble'
Master_dr = 'Master/Dr'
soldier = 'Soldier'
religious = 'Religious'

train['Title'] = train['Name'].str.extract('([A-Za-z]*)\.')
recode = { 'Miss' : no, 'Mlle' : no, 'Ms' : noble, 'Mrs' : noble, 'Mr' : noble, 'Sir' : noble, 'Jonkheer' : noble, 'Master' : Master_dr, 'Don' : noble, 'Rev' : religious, 'Dr' : Master_dr, 'Mme' : no, 'Major' : soldier, 'Lady' : noble, 'Col' : soldier, 'Capt' : soldier, 'Countess' : noble}
train = train.replace({'Title': recode})
title = pd.get_dummies(train['Title'])
train = pd.merge(train, title, left_index=True, right_index=True)

kaggle['Title'] = kaggle['Name'].str.extract('([A-Za-z]*)\.')
recode = { 'Miss' : no, 'Mrs' : noble, 'Mr' : noble, 'Master' : Master_dr, 'Ms' : noble, 'Col' : soldier, 'Rev' : religious, 'Dr' : Master_dr, 'Dona' : noble }
kaggle = kaggle.replace({'Title': recode})
title = pd.get_dummies(kaggle['Title'])
kaggle = pd.merge(kaggle, title, left_index=True, right_index=True)


/home/adou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  import sys
/home/adou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  del sys.path[0]


In [393]:
Support = train['SibSp'] + train['Parch']
train['Support'] = (Support > 0).astype('int')

Support = kaggle['SibSp'] + kaggle['Parch']
kaggle['Support'] = (Support > 0).astype('int')

In [394]:
mask = train['Embarked'].astype('str') == 'nan'
train['Embarked'].loc[mask] = 'S'

recode = {'S': 1, 'C' : 2, 'Q': 3}
train = train.replace({'Embarked' : recode})
kaggle = kaggle.replace({'Embarked' : recode})

/home/adou/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [395]:
correlation = train.corr()
correlation = correlation['Survived'].iloc[1:].abs().sort_values(ascending=False)
correlation

Sex                 0.543351
Pclass              0.338481
No Title            0.335636
Noble               0.335409
Cabin               0.316912
Pauvre              0.277673
riche               0.253865
+ Survived/Age      0.252817
+++ Survived/Age    0.230495
Support             0.203367
Embarked            0.106811
Master/Dr           0.082177
Parch               0.081629
Religious           0.064988
Age                 0.064909
++ Survived/Age     0.037451
SibSp               0.035322
moyen               0.026914
PassengerId         0.005007
Soldier             0.002496
Name: Survived, dtype: float64

In [396]:
cols = [ 'Survived','+ Survived/Age',
        '++ Survived/Age', '+++ Survived/Age', 'PassengerId',
        'Pclass', 'Sex', 'Cabin', 'Noble', 'Master/Dr',
        'Religious', 'Soldier', 'Pauvre', 'moyen', 'riche', 'Support']

data = train[cols]
data = data.set_index('PassengerId')

print(train.head())
cols = ['Parch', 'Age', 'SibSp', 'PassengerId', 'No Title', 'Embarked']
train = train.drop(cols, axis = 1)
kaggle = kaggle.drop(cols, axis = 1)

correlation = train.corr()
correlation = correlation['Survived'].index.values[1:]
X_cols = correlation
data.head()

             Survived  Pclass                      Name  Sex  Age  SibSp  \
PassengerId                                                                
1                   0       3   Braund, Mr. Owen Harris    0   22      1   
61                  0       3     Sirayanian, Mr. Orsen    0   22      0   
81                  0       3      Waelens, Mr. Achille    0   22      0   
113                 0       3    Barton, Mr. David John    0   22      0   
142                 1       3  Nysten, Miss. Anna Sofia    1   22      0   

             Parch     Ticket    Fare  Cabin   ...    Pauvre  moyen riche  \
PassengerId                                    ...                          
1                0  A/5 21171  Pauvre      1   ...         1      0     0   
61               0       2669  Pauvre      1   ...         1      0     0   
81               0     345767   moyen      1   ...         0      1     0   
113              0     324669  Pauvre      1   ...         1      0     0   
142  

,Survived,+ Survived/Age,++ Survived/Age,+++ Survived/Age,Pclass,Sex,Cabin,Noble,Master/Dr,Religious,Soldier,Pauvre,moyen,riche,Support
PassengerId,,,,,,,,,,,,,,,
1,0,0,1,0,3,0,1,1,0,0,0,1,0,0,1
61,0,0,1,0,3,0,1,1,0,0,0,1,0,0,0
81,0,0,1,0,3,0,1,1,0,0,0,0,1,0,0
113,0,0,1,0,3,0,1,1,0,0,0,1,0,0,0
142,1,0,1,0,3,1,1,0,0,0,0,1,0,0,0


In [397]:
X = train[X_cols]
X_kaggle = kaggle[X_cols]

y = train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X,y)


scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_kaggle = scaler.transform(X_kaggle)


c = [1, 100, 1000, 10000]

for c in c:
    LR = LogisticRegression(C=c)
    LR.fit(X_train, y_train)
    train_score = LR.score(X_train, y_train)
    test_score = LR.score(X_test, y_test)
    print("Pour c = %s. Training score = %s. Test score = %s" % (c, train_score, test_score))
    
X

Pour c = 1. Training score = 0.833832335329. Test score = 0.807174887892
Pour c = 100. Training score = 0.827844311377. Test score = 0.793721973094
Pour c = 1000. Training score = 0.827844311377. Test score = 0.793721973094
Pour c = 10000. Training score = 0.827844311377. Test score = 0.793721973094


,Pclass,Sex,Cabin,+++ Survived/Age,++ Survived/Age,+ Survived/Age,Pauvre,moyen,riche,Master/Dr,Noble,Religious,Soldier,Support
PassengerId,,,,,,,,,,,,,,
1,3,0,1,0,1,0,1,0,0,0,1,0,0,1
61,3,0,1,0,1,0,1,0,0,0,1,0,0,0
81,3,0,1,0,1,0,0,1,0,0,1,0,0,0
113,3,0,1,0,1,0,1,0,0,0,1,0,0,0
142,3,1,1,0,1,0,1,0,0,0,0,0,0,0
152,1,1,0,0,1,0,0,0,1,0,1,0,0,1
213,3,0,1,0,1,0,1,0,0,0,1,0,0,0
226,3,0,1,0,1,0,0,1,0,0,1,0,0,0
244,3,0,1,0,1,0,1,0,0,0,1,0,0,0


In [398]:
from sklearn.metrics import accuracy_score
def MLtr(X,y,c):
    i=0
    ac=0
    for el in range(1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=el)
        lr = LogisticRegression(C=c)
        lr.fit(X_train,y_train)
        y_pred=lr.predict(X_test)
        if accuracy_score(y_test, y_pred)>ac: 
            ac=accuracy_score(y_test, y_pred)
            i=el
    return (lr,i,ac)
X = train[X_cols]
X_kaggle = kaggle[X_cols]
X.head()
result=MLtr(X,y,10)

In [399]:
result[2]

0.8994413407821229

In [400]:
Survived = result[0].predict(X_kaggle)

data = pd.DataFrame( {'PassengerId' : PassengerId, 'Survived' : Survived} )
data = data.set_index('PassengerId')
data.to_csv('data.csv')

Tests des algos